In [24]:
from kfp.dsl import component, Output, Artifact, Input

@component(packages_to_install=['pandas'])
def testing(data: str, output1: Output[Artifact], output2: Output[Artifact]):
    import pandas as pd
    
    df = pd.DataFrame({"x": range(10)})
    
    with open(output1.path,'w') as f:
        f.write("hello world")
    
    df.to_csv(f"{output2.path}.csv")
    
@component(packages_to_install=["pandas"])
def receiver(input1: Input[Artifact], input2: Input[Artifact]) -> str:
    import pandas as pd
    
    with open(input1.path,'r') as f:
        x= f.read()
            
    df = pd.read_csv(f"{input2.path}.csv")
    print(df)
    return x

In [25]:
from kfp.dsl import pipeline

@pipeline(name='testing')
def pipeline():
    testing_job = testing(data="testing")
    recevier_job = receiver(
        input1=testing_job.outputs["output1"],
        input2=testing_job.outputs["output2"]
        )

In [26]:
from kfp import compiler

compiler.Compiler().compile(pipeline_func=pipeline, package_path="dummy.json")

In [27]:
from google.cloud import aiplatform


aiplatform.init(project="vtxdemos", location="us-central1")
job = aiplatform.PipelineJob(
        display_name="testing",
        template_path="dummy.json",
        pipeline_root="gs://vtxdemos-tmp",
        parameter_values={
        },
    )
job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/254356041555/locations/us-central1/pipelineJobs/testing-20230525142321
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/254356041555/locations/us-central1/pipelineJobs/testing-20230525142321')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/testing-20230525142321?project=254356041555
